### Quantum Volume - Qiskit Version (Adapted from IBM QV Tutorial and Paper)


In [1]:
min_qubits=2
max_qubits=6
max_circuits=1000
num_shots=10000

backend_id="qasm_simulator"
hub="ibm-q"; group="open"; project="main"
provider_backend = None

# # *** If using IBMQ hardware, run this once to authenticate
# from qiskit import IBMQ
# IBMQ.save_account('YOUR_API_TOKEN_HERE')

# # *** If you are part of an IBMQ group, set hub, group, and project name here
# hub="YOUR_HUB_NAME"
# group="YOUR_GROUP_NAME"
# project="YOUR_PROJECT_NAME"

# # *** This example shows how to specify the backend using a known "backend_id"
# backend_id="ibmq_belem"

# # *** Here's an example of using a typical custom provider backend (e.g. AQT simulator)
# import os
# from qiskit_aqt_provider import AQTProvider
# provider = AQTProvider(os.environ.get('AQT_ACCESS_KEY'))    # get your key from environment
# provider_backend = provider.backends.aqt_qasm_simulator_noise_1
# backend_id="aqt_qasm_simulator_noise_1"

# # An example using IonQ provider
# from qiskit_ionq import IonQProvider
# provider = IonQProvider()   # Be sure to set the QISKIT_IONQ_API_TOKEN environment variable
# provider_backend = provider.get_backend("ionq_qpu")
# backend_id="ionq_qpu"

# # *** Use these lines when running on hardware backend, to limit use of resources
# min_qubits=2
# max_qubits=5
# max_circuits=1
# num_shots=100

#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt

#Import Qiskit classes classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error

#Import the qv function.
import qiskit.ignis.verification.quantum_volume as qv

/var/folders/6z/0c4lt3293z7fq3ymhvchlgj80000gn/T/ipykernel_31566/2304083562.py:51: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  import qiskit.ignis.verification.quantum_volume as qv


In [2]:
# qv_circuits call requires connectivitiy to build circuits, will return QV circuits
# between min_qubits and max_qubits
qubit_lists = [list(range(i)) for i in range(min_qubits ,max_qubits+1)]
qv_circs, qv_circs_nomeas = qv.qv_circuits(qubit_lists, max_circuits)

In [3]:
# Add the results for perfect simulation to compare against
sim_backend = qiskit.Aer.get_backend('statevector_simulator')
ideal_results = []
for trial in range(max_circuits):
    # print('Simulating trial %d'%trial)
    ideal_results.append(qiskit.execute(qv_circs_nomeas[trial], backend=sim_backend, optimization_level=0).result())

qv_fitter = qv.QVFitter(qubit_lists=qubit_lists)
qv_fitter.add_statevectors(ideal_results)
    

In [5]:
## Running QV with our execute module

import sys
sys.path[1:1] = [ "_common", "_common/qiskit" ]
sys.path[1:1] = [ "../../_common", "../../_common/qiskit" ]
import execute as ex

exp_results = []

# Define custom result handler
def execution_handler(qc, result, num_qubits, s_int, num_shots):  
     
    exp_results.append(result)

def do_nothing_on_groups(group):
    pass

ex.init_execution(execution_handler)
ex.set_execution_target(backend_id, provider_backend=provider_backend,
            hub=hub, group=group, project=project)
ex.do_transpile_metrics = False # reduce runtime by not doing transpile metrics

if backend_id == 'qasm_simulator':
    # default noise model, can be overridden using set_noise_model
    noise = NoiseModel()
    # Add depolarizing error to all single qubit gates with error rate 0.3%
    one_qb_error = 0.003
    noise.add_all_qubit_quantum_error(depolarizing_error(one_qb_error, 1), ['rx', 'ry', 'rz'])

    # Add depolarizing error to all two qubit gates with error rate 3.0%
    two_qb_error = 0.03
    noise.add_all_qubit_quantum_error(depolarizing_error(two_qb_error, 2), ['cx'])

    ex.set_noise_model(noise)

for num_qubits in range(max_qubits - min_qubits + 1):
    for trial in range(max_circuits):
        # print('Running trial %d'%trial)
        ex.submit_circuit(qv_circs[trial][num_qubits], num_qubits, trial, shots=num_shots)
    ex.throttle_execution(do_nothing_on_groups)
    ex.finalize_execution(do_nothing_on_groups)

... execution complete at Jun 17, 2022 16:52:17 UTC in 1655484737.283 secs

... execution complete at Jun 17, 2022 17:09:10 UTC in 1655485750.91 secs

... execution complete at Jun 17, 2022 17:27:12 UTC in 1655486832.04 secs

... execution complete at Jun 17, 2022 17:45:22 UTC in 1655487922.283 secs



KeyboardInterrupt: 

In [ ]:
# add results to qv_fitter object
qv_fitter.add_data(exp_results)

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot the essence by calling plot_rb_data
qv_fitter.plot_qv_data(ax=ax, show_plt=False)

# Add title and label
ax.set_title('Quantum Volume for up to %d Qubits \n and %d Trials'%(len(qubit_lists[-1]), max_circuits), fontsize=18)

plt.show()

In [ ]:
qv_success_list = qv_fitter.qv_success()
qv_list = qv_fitter.ydata
for qidx, qubit_list in enumerate(qubit_lists):
    if qv_list[0][qidx]>2/3:
        if qv_success_list[qidx][0]:
            print("Width/depth %d greater than 2/3 (%f) with confidence %f (successful). Quantum volume %d"%
                  (len(qubit_list),qv_list[0][qidx],qv_success_list[qidx][1],qv_fitter.quantum_volume()[qidx]))
        else:
            print("Width/depth %d greater than 2/3 (%f) with confidence %f (unsuccessful)."%
                  (len(qubit_list),qv_list[0][qidx],qv_success_list[qidx][1]))
    else:
        print("Width/depth %d less than 2/3 (unsuccessful)."%len(qubit_list))
        

In [ ]:
# get ratios between QV depth and our transpiled depth, QV_transpile_factor

ratios = []
for i in range(len(qv_circs_nomeas)):
    for j in range(len(qv_circs_nomeas[i])):
        qc_depth = qv_circs_nomeas[i][j].decompose().depth()
        our_depth = qiskit.compiler.transpile(qv_circs_nomeas[i][j], basis_gates=['rx', 'ry', 'rz', 'cx']).depth()
        ratio = our_depth/qc_depth
        ratios.append(ratio)

QV_transpile_factor = np.mean(ratios)
print(f"Factor to increase QV depth by due to our transpilation target: {QV_transpile_factor}")